In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow.keras as keras

from tensorflow.keras.models import Model,load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
import tensorflow.keras.callbacks as kcallbacks
import os
import math
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import SGD
import operator
import numpy as np
from PIL import Image           # pip install pillow
from collections import defaultdict
import time
import cv2
import matplotlib.pyplot as plt
import time
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
# import tensorflow.keras as keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate,Dense,Flatten,Dropout,Average
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
import tensorflow.keras.callbacks as kcallbacks
import os
import math
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import SGD
import operator
import numpy as np
from PIL import Image
from collections import defaultdict
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import os
from sklearn.metrics import accuracy_score
# import tensorflow.keras as keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
import tensorflow.keras.callbacks as kcallbacks
import os
import math
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import SGD

In [2]:

TARGET_SIZE=(224,224)     
INPUT_SIZE=(224,224,3)    
BATCHSIZE=32           

test_datagen = ImageDataGenerator(rescale=1./255)

        
validation_generator = test_datagen.flow_from_directory(
            './test_shuffled_224/',
            target_size=TARGET_SIZE,
            batch_size=BATCHSIZE,
            class_mode='categorical')

Found 311 images belonging to 10 classes.


In [3]:
#generate labels indicating class
label=validation_generator.class_indices
label={v: k for k, v in label.items()}

In [4]:
 #load model 1: xception
xception_model = load_model('./xception.h5')
#load model 2: VGG16
vgg_model = load_model('./VGG16.h5')
#load model 3: VGG19
vgg19_model = load_model('./VGG19.h5')
#load model 4: inception
incep_model = load_model('./inception.h5')
#load model 5: inceptionresnet
inres_model = load_model('./inceptionresnet.h5')

In [6]:
model1=Model(inputs=[xception_model.layers[0].get_input_at(0)],outputs=xception_model.get_layer('dense_1').output,name='xception')
model2=Model(inputs=[vgg_model.layers[0].get_input_at(0)],outputs=vgg_model.get_layer('dense_3').output,name='vgg')
model3=Model(inputs=[vgg19_model.layers[0].get_input_at(0)],outputs=vgg19_model.get_layer('dense_5').output,name='vgg19')
model4=Model(inputs=[incep_model.layers[0].get_input_at(0)],outputs=incep_model.get_layer('dense_7').output,name='incep')
model5=Model(inputs=[inres_model.layers[0].get_input_at(0)],outputs=inres_model.get_layer('dense_9').output,name='inres')

In [7]:
def build_ensemble_model(num_class):
    img = Input(shape=(224, 224, 3), name='img')
    feature1 = model1(img)
    feature2 = model2(img)
    feature3 = model3(img)
    x = concatenate([feature1, feature2, feature3])
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.25)(x)
    output = Dense(num_class, activation='softmax', name='output')(x)
    model = Model(inputs=img, outputs=output)
    return model

def load_finished_ensemble_model():
    model=build_ensemble_model(num_class=10)
    model.load_weights('./ensemble.h5')
    return model

In [9]:
ensemble_model=load_model('./ensemble.h5')

In [10]:
#read images from validation folder


rootdir = './test_shuffled_224/'

test_laels = []
test_images=[]
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
            continue
        test_laels.append(subdir.split('/')[-1])
        test_images.append(os.path.join(subdir, file))
        
print(test_laels[0],test_images[0])

0 ./test_shuffled_224/0\11650.png


In [11]:
#test the averaging model on the validation set

predict=[]
length=len(test_images)
t1 = time.time()
for i in range((length//127)+1):
    inputimg=test_images[127*i:127*(i+1)]
    test_batch=[]
    for path in inputimg:
        thisimg=np.array(Image.open(path))/255
        test_batch.append(thisimg)
    #print(i, np.array(test_batch).shape)
    ensemble_model_batch=ensemble_model.predict(np.array(test_batch))
    predict_batch=list(np.argmax(ensemble_model_batch,axis=1))
    predict_batch=[label[con] for con in predict_batch]
    predict.append(predict_batch)

predict=sum(predict,[])

t2 = time.time()
print('The testing time is :%f seconds' % (t2-t1))

The testing time is :34.567217 seconds


In [12]:

acc=accuracy_score(test_laels,predict)
print('Concatenation accuracy:%s'%acc)

Concatenation accuracy:0.12540192926045016


In [13]:
acc=accuracy_score(test_laels,predict)
pre=precision_score(test_laels,predict,average='weighted')
re=recall_score(test_laels,predict,average='weighted')
f1=f1_score(test_laels,predict,average='weighted')
print('ensemble accuracy: %s'%acc)
print('precision: %s'%pre)
print('recall: %s'%re)
print('f1: %s'%f1)

print(confusion_matrix(test_laels, predict))
target_names = ['0', '1','2','3','4', '5', '6', '7', '8', '9']
print(classification_report(test_laels, predict, target_names=target_names))

ensemble accuracy: 0.12540192926045016
precision: 0.03193577126926179
recall: 0.12540192926045016
f1: 0.04666405607981778
[[ 0  0 11  0  0  0  0 25  0  0]
 [ 0  0 16  0  0  0  0 19  0  0]
 [ 0  0 14  0  0  0  0 23  0  0]
 [ 0  0  1  0  0  0  0 29  0  0]
 [ 0  0  6  0  0  0  0 23  0  0]
 [ 0  0  6  0  0  0  0 28  0  0]
 [ 0  0  7  0  0  0  0 26  0  0]
 [ 0  0  2  0  0  0  0 25  0  0]
 [ 0  0  9  0  0  0  0 19  0  0]
 [ 0  0  1  0  0  0  0 21  0  0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00        35
           2       0.19      0.38      0.25        37
           3       0.00      0.00      0.00        30
           4       0.00      0.00      0.00        29
           5       0.00      0.00      0.00        34
           6       0.00      0.00      0.00        33
           7       0.11      0.93      0.19        27
           8       0.00      0.00      0.00        28
      

In [14]:
label_map = {0: 'analysis', 1: 'backdoor', 2: 'dos', 3: 'exploits', 4: 'fuzzers', 5: 'generic', 6: 'normal', 7:'reconnaissance', 8:'shellcode', 9:'worms'}

In [15]:
y_true_list = [int(i) for i in test_laels]
y_pred_list = [int(i) for i in predict]

In [16]:
# Map the label values to their corresponding names using list comprehension
y_true_mapped = [label_map[y] for y in y_true_list]
y_pred_mapped = [label_map[y] for y in y_pred_list]

# Generate the classification report
print(classification_report(y_true_mapped, y_pred_mapped, zero_division=1))
print('\n')
print('                    Confusion Matrix')

report = classification_report(y_true_mapped, y_pred_mapped, zero_division=1, output_dict=True)

cm = confusion_matrix(y_true_mapped, y_pred_mapped)
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)

for class_label, acc in zip(report.keys(), per_class_accuracy):
    if class_label not in ['accuracy', 'macro avg', 'weighted avg']:
        print(f"Class {class_label} Accuracy: {acc * 100:.2f}%")

                precision    recall  f1-score   support

      analysis       1.00      0.00      0.00        36
      backdoor       1.00      0.00      0.00        35
           dos       0.19      0.38      0.25        37
      exploits       1.00      0.00      0.00        30
       fuzzers       1.00      0.00      0.00        29
       generic       1.00      0.00      0.00        34
        normal       1.00      0.00      0.00        33
reconnaissance       0.11      0.93      0.19        27
     shellcode       1.00      0.00      0.00        28
         worms       1.00      0.00      0.00        22

      accuracy                           0.13       311
     macro avg       0.83      0.13      0.04       311
  weighted avg       0.83      0.13      0.05       311



                    Confusion Matrix
Class analysis Accuracy: 0.00%
Class backdoor Accuracy: 0.00%
Class dos Accuracy: 37.84%
Class exploits Accuracy: 0.00%
Class fuzzers Accuracy: 0.00%
Class generic Accuracy: 

In [17]:
y_true_mapped = [label_map[y] for y in y_true_list]
y_pred_mapped = [label_map[y] for y in y_pred_list]

report = classification_report(y_true_mapped, y_pred_mapped, zero_division=1, output_dict=True)

cm = confusion_matrix(y_true_mapped, y_pred_mapped)
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)

for label, metrics in report.items():
    if isinstance(metrics, dict):  # skip accuracy, macro avg, etc.
        print(f"\nClass: {label}")
        for metric_name, value in metrics.items():
            print(f"  {metric_name}: {value:.3f}")

for class_label, acc in zip(report.keys(), per_class_accuracy):
    if class_label not in ['accuracy', 'macro avg', 'weighted avg']:
        print(f"Class {class_label} Accuracy: {acc * 100:.3f}%")
        


Class: analysis
  precision: 1.000
  recall: 0.000
  f1-score: 0.000
  support: 36.000

Class: backdoor
  precision: 1.000
  recall: 0.000
  f1-score: 0.000
  support: 35.000

Class: dos
  precision: 0.192
  recall: 0.378
  f1-score: 0.255
  support: 37.000

Class: exploits
  precision: 1.000
  recall: 0.000
  f1-score: 0.000
  support: 30.000

Class: fuzzers
  precision: 1.000
  recall: 0.000
  f1-score: 0.000
  support: 29.000

Class: generic
  precision: 1.000
  recall: 0.000
  f1-score: 0.000
  support: 34.000

Class: normal
  precision: 1.000
  recall: 0.000
  f1-score: 0.000
  support: 33.000

Class: reconnaissance
  precision: 0.105
  recall: 0.926
  f1-score: 0.189
  support: 27.000

Class: shellcode
  precision: 1.000
  recall: 0.000
  f1-score: 0.000
  support: 28.000

Class: worms
  precision: 1.000
  recall: 0.000
  f1-score: 0.000
  support: 22.000

Class: macro avg
  precision: 0.830
  recall: 0.130
  f1-score: 0.044
  support: 311.000

Class: weighted avg
  precision: 0

In [18]:
print(classification_report(y_true_mapped, y_pred_mapped, zero_division=1))
print('\n')
print('                    Confusion Matrix')

                precision    recall  f1-score   support

      analysis       1.00      0.00      0.00        36
      backdoor       1.00      0.00      0.00        35
           dos       0.19      0.38      0.25        37
      exploits       1.00      0.00      0.00        30
       fuzzers       1.00      0.00      0.00        29
       generic       1.00      0.00      0.00        34
        normal       1.00      0.00      0.00        33
reconnaissance       0.11      0.93      0.19        27
     shellcode       1.00      0.00      0.00        28
         worms       1.00      0.00      0.00        22

      accuracy                           0.13       311
     macro avg       0.83      0.13      0.04       311
  weighted avg       0.83      0.13      0.05       311



                    Confusion Matrix
